In [87]:
import numpy as np

from sklearn.base import BaseEstimator

from sklearn.utils.testing import assert_array_equal
from sklearn.utils.testing import assert_array_almost_equal
from sklearn.utils.testing import assert_equal
from sklearn.utils.testing import assert_raises
from sklearn.utils.testing import assert_greater
from sklearn.utils.testing import assert_less
from sklearn.utils.testing import assert_true
from sklearn.utils.testing import assert_false
from sklearn.utils.testing import assert_warns
from sklearn.utils.testing import assert_warns_message

from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, SVR, LinearSVC
from sklearn.naive_bayes import GaussianNB


from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston, load_iris, make_hastie_10_2
from sklearn.utils import check_random_state

from scipy.sparse import csc_matrix, csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import csv 
import pandas
import time


start = time.time()
rng = check_random_state(0)

#Loading the data
header_reader = csv.reader(open("./Matlab_Files/joined_shots_2013.csv",'r'),delimiter = ',')
headers = set(next(header_reader))
#print (headers)

#main_reader = csv.reader(open("./Matlab_Files/joined_shots_updated2013.csv",'r'),delimiter = ',')
dataframe = pandas.read_csv("./Matlab_Files/joined_shots_updated2013.csv",delimiter = ',',header =None)
array = dataframe.values

column_len = len(dataframe.columns)
columns = dataframe.columns

X = array[0:200961,0:30]
Y = array[0:200961,31]


# Check classification for various parameter settings.
rng = check_random_state(0)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    Y,
                                                    random_state=rng)

In [88]:
def Stacking(model,train,y,test,n_fold):
   folds=StratifiedKFold(n_splits=n_fold,random_state=1)
   test_pred=np.empty((test.shape[0],1),float)
   train_pred=np.empty((0,1),float)
   for train_indices,val_indices in folds.split(train,y):
      x_train,x_val=train[train_indices],train[val_indices]
      y_train,y_val=y[train_indices],y[val_indices]

      model.fit(X=x_train,y=y_train)
      train_pred=np.append(train_pred,model.predict(x_val))
      test_pred=np.append(test_pred,model.predict(test))
   return test_pred.reshape(-1,1),train_pred


In [89]:
model1 = DecisionTreeClassifier(random_state=1)

test_pred1 ,train_pred1=Stacking(model=model1, train=X_train,y=y_train,test=X_test,n_fold=10)

train_pred1=pandas.DataFrame(train_pred1)
test_pred1=pandas.DataFrame(test_pred1)

In [97]:
model2 = KNeighborsClassifier()

test_pred2 ,train_pred2=Stacking(model=model2,train=X_train,y=y_train,test=X_test,n_fold=10)

train_pred2=pandas.DataFrame(train_pred2)
test_pred2=pandas.DataFrame(test_pred2)

In [91]:
model3 = QuadraticDiscriminantAnalysis()

test_pred3 ,train_pred3=Stacking(model=model3,train=X_train,y=y_train,test=X_test,n_fold=10)

train_pred3=pandas.DataFrame(train_pred3)
test_pred3=pandas.DataFrame(test_pred3)

C:\Users\mcokb\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [94]:
temp=y_test
for x in range(0, 10):
    y_test=np.append(y_test,temp)


In [98]:
df = pandas.concat([train_pred1, train_pred2, train_pred3], axis=1)
df_test = pandas.concat([test_pred1, test_pred2, test_pred3], axis=1)

model = LogisticRegression(random_state=1)
model.fit(df,y_train)
model.score(df_test, y_test)

0.6111813784829847

In [58]:
print(y_test.shape)

(50241,)


In [74]:
print(y_test.shape)


(602892,)
